# Clusteranalyse mit GMM

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns    
from sklearn.mixture import GaussianMixture as GMM
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [ ]:
def Musterplot(ax, title, xlabel, ylabel, xticks, yticks):
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)

    ax.xaxis.set_tick_params(top='on', direction='in', width=1)
    ax.yaxis.set_tick_params(right='on', direction='in', width=1)
    
    ax.set_title(title,fontweight='bold',fontsize=14)
    
    ax.set_xlabel(xlabel,fontweight='bold',fontsize=12)
    ax.set_ylabel(ylabel,fontweight='bold',fontsize=12)
    
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)

In [ ]:
df = pd.read_csv("Datensatz_Clusteranalyse.csv", sep=";")

In [ ]:
df2 = df[['Ankunftszeit','Abfahrtszeit','Ladeanteil','Leistung_max (kW)','Verbrauch (kWh)']]

In [ ]:
## Skalierung der Merkmale 

scaler = StandardScaler()
X = scaler.fit_transform(df2)
X = pd.DataFrame(X, columns=df2.columns)

In [ ]:
## Ermittung von BIC und AIC für die Bestimmung der Clusteranzahl (k)

bic = []
aic = []

for k in range(2, 12):
    gmm = GMM(n_components=k, init_params='k-means++', random_state=None)
    gmm.fit(X)
    prediction_gmm = gmm.predict(X)
    bic.append(gmm.bic(X))
    aic.append(gmm.aic(X))  
    
fig, ax = plt.subplots()

xlabel = "k: Anzahl der Cluster"
ylabel = "BIC/AIC Wert"
title = "BIC und AIC"

yticks = np.arange(6000, 16000, step=1000)
xticks = np.arange(2, 13, step=1)
plt.grid()
ax.plot(range(2, 12), bic, color='blue', marker='o',label='bic')
ax.plot(range(2, 12), aic, color='orange', marker='o',label='aic')
plt.legend(loc='upper right')
Musterplot(ax, title, xlabel, ylabel, xticks, yticks)

plt.show()

# fig.savefig("BIC_AIC_L_V")    


In [ ]:
## Berechnung von AIC für 500 Initialisierung von GMM

aic = []

for k in range(1, 501):
    gmm = GMM(n_components = 6, init_params = 'k-means++', random_state = k).fit(X)
    prediction_gmm = gmm.predict(X)
    aic.append(gmm.aic(X))
    
erg = pd.DataFrame(aic,  columns=['aic'])    

In [ ]:
## Initialisierung mit niedrigstem AIC wird ausgewählt 

aic_min = erg.iloc[erg['aic'].idxmin()]

In [ ]:
## Beste Initialisierung von GMM über random_state

random_state = aic_min.name

gmm = GMM(n_components = 6, init_params = 'k-means++', random_state = random_state).fit(X)
prediction_gmm = gmm.predict(X)

In [ ]:
# Statistische Analyse der Cluster durch Mittelwert

df2['Labels'] = prediction_gmm
df2['Cluster'] = df2['Labels'].map({0:'Eins', 1:'Zwei', 2:'Drei',3:'Vier',4:'Fünf',5:'Sechs'})
df2['Cluster'] = df2['Cluster'].astype('category')
df2['Cluster'] = df2['Cluster'].cat.reorder_categories(['Eins','Zwei','Drei','Vier','Fünf','Sechs'])

df3 = df2.groupby('Cluster').agg(
    { 
        'Labels':'count',
        'Ankunftszeit': 'mean',
        'Abfahrtszeit': 'mean',
        'Ladeanteil':'mean',
        'Leistung_max (kW)': 'mean',
        'Verbrauch (kWh)': 'mean',
    }
).reset_index()

In [ ]:
df3

In [ ]:
## Plot für zweidimensionales Beispiel (Ankunfts- und Abfahrtszeit)

# fig, ax = plt.subplots(1,1,figsize=(7,5))

# xlabel = "Ankunftszeit in h"
# ylabel = "Abfahrtszeit in h "
# title = "Clusterergebnis mit GMM (Ankunfts- und Abfahrtszeit)"
# yticks = np.arange(5, 25, step=1)
# xticks = np.arange(5, 25, step=1)
# plt.grid()
# ax.set_axisbelow(True)
# sns.scatterplot(df2['Ankunftszeit'], df2['Abfahrtszeit'], hue = df2['Labels'],palette="deep")
# # ax.scatter(df['Ankunftszeit'],df['Abfahrtszeit'], c = df2['Labels'])
# ax.scatter(gmm.means_[:,0] , gmm.means_[:,1] ,marker='x', s = 200, color = 'k')
# Musterplot(ax, title, xlabel, ylabel, xticks, yticks)

# plt.show()

# fig.savefig("Scatterplot_AN_AB_Beispiel_GMM")

In [ ]:
## Plot für Zweidimensionales Beispiel (max. Leistung und Gesamtsverbrauch)

# X['Labels'] = prediction_gmm
# fig, ax = plt.subplots(1,1,figsize=(7,5))

# xlabel = "max. Leistung (skaliert)"
# ylabel = "Verbrauch (skaliert)"
# title = "Clusterergebnis mit GMM (max. Leistung und Verbrauch)"
# yticks = np.arange(-2, 5.1, step=0.4)
# xticks = np.arange(-2, 4, step=0.4)
# plt.grid()
# ax.set_axisbelow(True)
# sns.scatterplot(X['Leistung_max (kW)'], X['Verbrauch (kWh)'], hue = X['Labels'], palette="deep")
# # ax.scatter(df['Ankunftszeit'], df['Abfahrtszeit'], c = df2['Labels'])
# ax.scatter(gmm.means_[:,0] , gmm.means_[:,1] , marker='x', s = 200, color = 'k')
# Musterplot(ax, title, xlabel, ylabel, xticks, yticks)

# plt.show()

# fig.savefig("Scatterplot_L_V_Beispiel_GMM", dpi=600)

In [ ]:
## Boxplot für die Interpretation der Cluster 

fig, ax = plt.subplots(1,1,figsize=(5,7))

xlabel = "Cluster"
ylabel = "Merkmal"
title = "Boxplot: "

sns.boxplot(x='Cluster', y='Merkmal',data=df2, palette="deep",
                    medianprops={"color": "black"}, width=0.5, showmeans=True, meanprops={"marker":"_",
                                                                                         "markerfacecolor":"red",
                                                                                         "markersize": "20",
                                                                                         "markeredgecolor":"red"})
plt.grid()
ax.set_axisbelow(True)
ax.set_title(title)
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
ax.spines['top'].set_visible(True)
ax.spines['right'].set_visible(True)
ax.xaxis.set_tick_params(top='on', direction='in', width=1)
ax.yaxis.set_tick_params(right='on', direction='in', width=1)
ax.set_title(title,fontweight='bold',fontsize=14)
ax.set_xlabel(xlabel,fontweight='bold',fontsize=12)
ax.set_ylabel(ylabel,fontweight='bold',fontsize=12)

plt.show()

# fig.savefig("Boxplot_Merkmal")